In [9]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tf.constant(ids)

print(input_ids)

model(input_ids)

c:\Users\user\.conda\envs\ds-vl2\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tf.Tensor(
[ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
  2166  1012], shape=(14,), dtype=int32)


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.7276213,  2.8789363]], dtype=float32)>, hidden_states=None, attentions=None)

In [10]:
tokenized_inputs = tokenizer(sequence, return_tensors="tf")
print(tokenized_inputs["input_ids"])

tf.Tensor(
[[  101  1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026
   2878  2166  1012   102]], shape=(1, 16), dtype=int32)


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

# Последовательность действий
# 1. Перобразовываем в токены
tokens = tokenizer.tokenize(sequence)
print('TOKENS:' ,tokens)
# 2. Получаем ID токенов
ids = tokenizer.convert_tokens_to_ids(tokens)
print('IDS:', ids)

# 3. Этот шаг преобразует список числовых идентификаторов токенов (ids) в тензор TensorFlow
input_ids = tf.constant([ids])
print("Input IDs:", input_ids)

# 4. Передаем подготовленные данные в обученную модель для получения предсказания
output = model(input_ids)
print("Logits:", output.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


TOKENS: ['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']
IDS: [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]
Input IDs: tf.Tensor(
[[ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
   2166  1012]], shape=(1, 14), dtype=int32)
Logits: tf.Tensor([[-2.7276213  2.8789363]], shape=(1, 2), dtype=float32)


Батчинг - это отправка нескольких предложений через модель одновременно. Если у вас есть только одно предложение, вы можете просто создать батч с одной последовательностью:

In [12]:
batched_ids = [ids, ids]

input_ids = tf.constant(batched_ids)
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tf.Tensor(
[[ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
   2166  1012]
 [ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
   2166  1012]], shape=(2, 14), dtype=int32)
Logits: tf.Tensor(
[[-2.7276204  2.8789377]
 [-2.7276204  2.8789377]], shape=(2, 2), dtype=float32)


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# checkpoint — это имя конкретной модели
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# загружает токенизатор, который соответствует выбранной модели. Он знает, как преобразовать текст в последовательности чисел (токенов)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# загружает саму модель, которая будет классифицировать текст, работая в среде TensorFlow.
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1 = "I've been waiting for a HuggingFace course my whole life."
sequence2 = "I hate this so much!"

#[sequence1, sequence2] — вы передаете обе строки в виде списка. Это позволяет токенизатору обрабатывать их вместе как один "пакет" (batch).
# padding=True — этот аргумент указывает токенизатору, что он должен дополнить более короткую последовательность специальными токенами-заполнителями ([PAD]), 
# чтобы она имела ту же длину, что и самая длинная в пакете.
# truncation=True - обрезание строки
# return_tensors='tf' — этот аргумент сообщает токенизатору, что результат должен быть возвращен в виде тензоров TensorFlow, а не обычных списков Python.
# В результате tokenizer_sequences становится словарем, содержащим тензор input_ids (идентификаторы токенов) и attention_mask (маску внимания).
tokenizer_sequences = tokenizer([sequence1, sequence2], padding=True, return_tensors='tf')
print("tokenizer_sequences:", tokenizer_sequences['input_ids'])

# Здесь вы передаете весь словарь tokenizer_sequences в модель. 
# Модель принимает его и автоматически использует как input_ids, так и attention_mask. 
# Маска внимания нужна для того, чтобы модель игнорировала токены-заполнители, которые были добавлены на предыдущем шаге.
output = model(tokenizer_sequences)
print("Logits:", output.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_sequences: tf.Tensor(
[[  101  1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026
   2878  2166  1012   102]
 [  101  1045  5223  2023  2061  2172   999   102     0     0     0     0
      0     0     0     0]], shape=(2, 16), dtype=int32)
Logits: tf.Tensor(
[[-1.5606993  1.6122842]
 [ 4.169231  -3.3464477]], shape=(2, 2), dtype=float32)
